### Reinforcement Learning Example

In this notebook, we implement a very simple model based on the Q-learning algorithm. This notebook is intend to show a basic form of RL that doesn't require a deep understanding of neural networks or advanced mathematics and how one might deploy such a model.

This example shows the Grid World problem, where an agent learns to navigate a grid to reach a goal.

The notebook will go through the following steps:

1. Define State and Action Space
2. Create a Q-table to store expected rewards for each state/action combination
3. Implement learning algorithm and train model
4. Evaluate model

### 1. Define State and Action Space

In [1]:
import numpy as np
import random

In [2]:
# Grid settings
grid_size = 4

#funtion to build list of all state tuples
def build_state_list(grid_size):
    state_list = []
    for i in range(grid_size):
        for j in range(grid_size):
            state_list.append((i, j))
    return state_list
all_states = build_state_list(grid_size)

# Here we just try to reach the top right corner (could be center or any other state)
goal_state = (3, 3)
n_states = grid_size * grid_size
n_actions = 4  # Up, Down, Left, Right

### 2. Create a Q-table to store expected rewards for each state/action combination

In [3]:
# Initialize Q-table
Q = np.zeros((n_states, n_actions))

# Helper functions
def state_to_index(state):
    return state[0] * grid_size + state[1]

def index_to_state(index):
    return (index // grid_size, index % grid_size)

def get_possible_actions(state):
    actions = []
    if state[0] > 0: actions.append(0)  # Up
    if state[0] < grid_size - 1: actions.append(1)  # Down
    if state[1] > 0: actions.append(2)  # Left
    if state[1] < grid_size - 1: actions.append(3)  # Right
    return actions

# Correct the state transition function to prevent invalid states
def take_action(state, action):
    new_state = list(state)
    if action == 0 and state[0] > 0: new_state[0] -= 1  # Up
    if action == 1 and state[0] < grid_size - 1: new_state[0] += 1  # Down
    if action == 2 and state[1] > 0: new_state[1] -= 1  # Left
    if action == 3 and state[1] < grid_size - 1: new_state[1] += 1  # Right
    return tuple(new_state)

### 3. Implement learning algorithm and train model

In [4]:
# Learning parameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1  # Exploration rate
n_episodes = 100000

# Training the model with corrected state transitions
for episode in range(n_episodes):
    #start at a random state
    state = random.choice(all_states)
    done = state == goal_state

    while not done:
        state_index = state_to_index(state)
        if random.uniform(0, 1) < epsilon:
            # Explore: choose a random action
            action = random.choice(get_possible_actions(state))
        else:
            # Exploit: choose the best action from Q-table
            action = np.argmax(Q[state_index])

        # Take action and observe reward
        next_state = take_action(state, action)
        reward = 1 if next_state == goal_state else 0
        next_state_index = state_to_index(next_state)

        # Q-learning update
        Q[state_index, action] = (Q[state_index, action] + 
                                  learning_rate * (reward + 
                                  discount_factor * np.max(Q[next_state_index]) - 
                                  Q[state_index, action]))

        # Transition to the next state
        state = next_state
        done = state == goal_state

### 4. Evaluate model

First, we will just show one path then see on average how many actions it takes to get to the goal state

In [5]:
# Evaluating the model
state = random.choice(all_states)
print("Initial state:", state)
trajectory = [state]
done = state == goal_state
while not done:
    state_index = state_to_index(state)
    action = np.argmax(Q[state_index])  # Choose the best action
    state = take_action(state, action)
    trajectory.append(state)
    done = state == goal_state

print(trajectory)

Initial state: (2, 0)
[(2, 0), (2, 1), (3, 1), (3, 2), (3, 3)]


In [6]:
total_actions = 0 # Total number of actions taken to reach the goal
for state in all_states:
    # Evaluating the model
    trajectory = [state]
    done = state == goal_state
    while not done:
        state_index = state_to_index(state)
        action = np.argmax(Q[state_index])  # Choose the best action
        state = take_action(state, action)
        trajectory.append(state)
        done = state == goal_state
        total_actions += 1
print("Average number of actions taken to reach the goal:", total_actions / len(all_states))
        

Average number of actions taken to reach the goal: 3.0


Is this optimal? We know the optimal policy is to move up or to the right until we reach the goal. From the bottom left, this is 6 actions, for the next 2 states it is 5 actions, for the next 3 it is 4 actions, then 4->3, 3->2, 2->1, 1->0 as we already start at the goal state. By simple arithmetic we have

6+2\*5+3\*4+4\*3+3\*2+2\*1 = 48

Total state = 16

Therefore, the optimal is 48/16 = 3 which is exactly our average number of actions.